In [ ]:
pip install pandas scikit-learn sentence-transformers

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv"
df = pd.read_csv(url, sep="\t", header=None, names=["label", "text"])
print(len(df), "rows loaded")


5572 rows loaded


In [ ]:
chunks = []
chunk_size = 10   # 12 chunks * 10 = 120
for i in range(0, len(df_small), chunk_size):
    chunk_text = " ".join(df_small["text"][i:i+chunk_size])
    chunks.append(chunk_text)

print("Created chunks:", len(chunks))   # expect 12

# Print all 12 chunks
for i, c in enumerate(chunks):
    print(f"\n------ Chunk {i+1} ------")
    print("Length:", len(c))
    print("Content:\n", c)


Created chunks: 12

------ Chunk 1 ------
Length: 862
Content:
 Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat... Ok lar... Joking wif u oni... U dun say so early hor... U c already then say... Nah I don't think he goes to usf, he lives around here though Even my brother is not like to speak with me. They treat me like aids patent. As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today. I've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times. I HAVE A DATE ON SUNDAY WITH WILL!! Oh k...i'm watching here:)

------ Chunk 2 ------
Length: 739
Content:
 Eh u remember how 2 spell hi

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')   # loads model

texts = chunks  # list of chunk strings
embeddings = model.encode(texts, show_progress_bar=True)  # shape: (12, dim)
print("Embeddings shape:", embeddings.shape)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Embeddings shape: (12, 384)


In [ ]:
labels = []
for i in range(0, len(df_small), chunk_size):
    part = df_small["label"][i:i+chunk_size]
    spam = (part == "spam").sum()
    ham = (part == "ham").sum()
    labels.append(1 if spam > ham else 0)

print("Chunk labels:", labels)   # e.g. [0,0,0,0,0,0,1,1,1,1,1,1]


Chunk labels: [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]


In [ ]:
import numpy as np

# choose first 10 chunks for training, last 2 for testing (as before)
X = embeddings
y = np.array(labels)

X_train = X[:10]
y_train = y[:10]

X_test = X[10:]
y_test = y[10:]

print("Train size:", X_train.shape[0], "Test size:", X_test.shape[0])
print("y_test:", y_test.tolist())


Train size: 10 Test size: 2
y_test: [1, 1]


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

k_values = [1,2,3,4,5,6,7,8,9,10]   # valid k <= number of training samples
results = {}

for k in k_values:
    if k > len(X_train):
        continue
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    preds = knn.predict(X_test)
    results[k] = preds.tolist()

# print predictions per k
for k, preds in results.items():
    print(f"k={k} -> preds={preds}")


k=1 -> preds=[1, 1]
k=2 -> preds=[1, 1]
k=3 -> preds=[1, 1]
k=4 -> preds=[1, 1]
k=5 -> preds=[1, 1]
k=6 -> preds=[1, 1]
k=7 -> preds=[1, 1]
k=8 -> preds=[0, 0]
k=9 -> preds=[0, 0]
k=10 -> preds=[0, 0]


In [ ]:
#“The difference in 0,0 and 1,1 occurs because some chunks contain mostly messages from class 0 (HAM) and some chunks mostly from class 1 (SPAM).
# The label reflects the majority in that chunk, and KNN predicts based on the nearest neighbors’ class.”
#1,1 → the sentences are similar
#0,0 → the sentences are dissimilar


In [ ]:
#KNN predicts based on neighbors

#If neighbors are mostly HAM → prediction = 0

#If neighbors are mostly SPAM → prediction = 1

#Hence predictions can be 0,0 or 1,1 depending on chunk content